Test shipping

In [1]:
import sys
import os
import json
import logging
from kafka import KafkaConsumer, KafkaProducer
from json import loads
import random
import datetime
import mysql.connector
import boto3
from logging.handlers import RotatingFileHandler

In [4]:
def accounting_consumer():
    # Configure logging
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.ERROR)

    mysql_conn = mysql.connector.connect(
        host='localhost',
        user='mysql',
        password='mysql',
        database='oaken'
    )

    mysql_cursor = mysql_conn.cursor()

    # Kafka consumers
    shipping_consumer = KafkaConsumer(
        'shipping',
        bootstrap_servers=['localhost:9092'],
        auto_offset_reset='earliest',  # Start consuming from the earliest offset
        enable_auto_commit=True,       # Automatically commit offsets
        group_id='oaken_accounting_group',  # Specify a consumer group
        value_deserializer=lambda x: loads(x.decode('utf-8')))

    shipping_consumer.subscribe(topics='shipping')

    for shipping_message in shipping_consumer:
        try:
            shipping_data = shipping_message.value

            invoice = shipping_data.get('Invoice', '')

            shipping_expense = float(shipping_data.get('ShippingCost')) * -1

            sales = float(shipping_data.get('SaleDollars'))

            # MySQL
            try:
                LEDGER_CREDIT = """
                    INSERT INTO salesLedger (Invoice, Credit, Note)
                    VALUES (%s, %s, 'Sale')
                """
                credit_data = (invoice, sales)
                mysql_cursor.execute(LEDGER_CREDIT, credit_data)
                mysql_conn.commit()
            except Exception as e:
                logger.error(f"Error processing message: {e}")
                pass

            try:
                LEDGER_DEBIT = """
                    INSERT INTO salesLedger (Invoice, Debit, Note)
                    VALUES (%s, %s, 'Shipping')
                """
                debit_data = (invoice, shipping_expense)
                mysql_cursor.execute(LEDGER_DEBIT, debit_data)
                mysql_conn.commit()
            except Exception as e:
                logger.error(f"Error processing message: {e}")
                pass

        except Exception as e:
            logger.error(f"Error processing message: {e}")
            pass

    mysql_cursor.close()
    mysql_conn.close()

In [5]:
accounting_consumer()

NameError: name 'logger' is not defined

In [ ]:
# sample output
'''
{'Invoice/Item Number': 'S29198800001', 'Date': '11/20/2015', 'Store Number': '2191', 'Store Name': 'Keokuk Spirits', 
'Address': '1013 MAIN', 'City': 'KEOKUK', 'Zip Code': '52632', 'Store Location': '1013 MAIN\nKEOKUK 52632\n(40.39978, -91.387531)', 
'County Number': '56', 'County': 'Lee', 'Category': '', 'Category Name': '', 'Vendor Number': '255', 'Vendor Name': 'Wilson Daniels Ltd.', 
'Item Number': '297', 'Item Description': 'Templeton Rye w/Flask', 'Pack': '6', 'Bottle Volume (ml)': '750', 'State Bottle Cost': '$18.09', 
'State Bottle Retail': '$27.14', 'Bottles Sold': '6', 'Sale (Dollars)': '$162.84', 'Volume Sold (Liters)': '4.50', 'Volume Sold (Gallons)': '1.19'}
'''